In [1]:
import boto3
import json
from tqdm import tqdm
import pandas as pd
import numpy as np
from io import BytesIO
import ast
import random
import PIL.Image as Image
import time

from functools import partial
from aquabyte.lib.db.snowflake import snowflake_query_to_df
SNOWFLAKE_DSN = '/dsn/snowflake/mochi'
snowflake_query_to_df = partial(snowflake_query_to_df, ssm_name=SNOWFLAKE_DSN)

import os

In [2]:
ROOT_DIR = "/workspace/mnt/"
TRAIN_SAVE_PATH = f"{ROOT_DIR}data/train2017/"
VAL_SAVE_PATH = f"{ROOT_DIR}data/val2017/"
ANNOTATION_SAVE_PATH = f"{ROOT_DIR}data/annotations/"
VAL_FRAC = 0.1
os.makedirs(TRAIN_SAVE_PATH, exist_ok=True)
os.makedirs(VAL_SAVE_PATH, exist_ok=True)
os.makedirs(ANNOTATION_SAVE_PATH, exist_ok=True)


# these constants are used to load the dataset used to train the previous model
OLD_BUCKET = "s3://aquabyte-frames-resized-inbound/"
NEW_BUCKET = "s3://aquabyte-research/pwais/mft-pg/datasets_s3/high_recall_fish1/images/"
S3_CSV_PATH = "s3://aquabyte-research/pwais/mft-pg/datasets_s3/high_recall_fish1/hrf_with_keypoint_visibility_dataset.csv"

category_id_to_name_json = '{"0": "HIGH", "1": "LOW", "2": "MEDIUM", "3": "PARTIAL"}' 
id_to_category = json.loads(category_id_to_name_json)
category_to_id = dict((c, i) for i, c in id_to_category.items())

In [3]:
# # use boto to load a csv file from s3
s3 = boto3.client('s3')
bucket, key = S3_CSV_PATH[5:].split('/', 1)
obj = s3.get_object(Bucket=bucket, Key=key)
df_hr_dataset = pd.read_csv(BytesIO(obj['Body'].read()))
# load the literals using ast for the following columns: images, metadata, label_set, original_annotation
df_hr_dataset['images'] = df_hr_dataset['images'].apply(lambda x: ast.literal_eval(x)[0])
df_hr_dataset['metadata'] = df_hr_dataset['metadata'].apply(lambda x: ast.literal_eval(x))
df_hr_dataset['label_set'] = df_hr_dataset['label_set'].apply(lambda x: ast.literal_eval(x))
df_hr_dataset['original_annotation'] = df_hr_dataset['original_annotation'].apply(lambda x: ast.literal_eval(x))
df_hr_dataset['annotation'] = df_hr_dataset['annotation'].apply(lambda x: ast.literal_eval(x))
df_hr_dataset['pen_id'] = df_hr_dataset['images'].apply(lambda x: x.split('/pen-id=')[1].split('/')[0])
df_hr_dataset['captured_at'] = df_hr_dataset['images'].apply(lambda x: x.split('/at=')[1].split('/')[0])

# replace the image path with the new bucket in images
df_hr_dataset['images'] = df_hr_dataset['images'].apply(lambda x: x.replace(OLD_BUCKET, NEW_BUCKET))

#   drop columns starting with "Unnamed"
df_hr_dataset = df_hr_dataset.loc[:, ~df_hr_dataset.columns.str.contains('^Unnamed')]
df_hr_dataset

,id,plali_image_id,original_annotation,annotator_email,plali_image_id.1,images,metadata,workflow_id,name,label_set,full_count,full_and_partial_count,annotation,path_on_quad,pen_id,captured_at
0,1865758d-36c7-4422-a408-a4a5c0dadfd6,7ec50a97-7ef8-43e6-851f-0bbcca5a8bb2,"{'isPartial': True, 'annotations': [{'label': ...",labeler41@cogitotech.com,7ec50a97-7ef8-43e6-851f-0bbcca5a8bb2,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['medium_turbidity'], 'crops': [{'id'...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",2,12,"[{'label': 'HIGH', 'width': 408, 'xCrop': 54, ...",/data8tb/biomass-retrain/Image00000.jpg,56,2020-07-06T13:23:13.686606000Z
1,b58fb82e-23bf-4329-a333-2f470d2bc8ba,501bc9e3-c6da-4295-8277-c147ca15a4cf,"{'isPartial': True, 'annotations': [{'label': ...",labeler39@cogitotech.com,501bc9e3-c6da-4295-8277-c147ca15a4cf,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['medium_turbidity'], 'crops': [{'id'...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",2,6,"[{'label': 'PARTIAL', 'width': 177, 'xCrop': 2...",/data8tb/biomass-retrain/Image00001.jpg,56,2020-07-09T08:17:06.626281000Z
2,239df62a-664e-40ba-ba3b-0b0ac04efb85,8a1a3d87-8a2d-47e5-8670-337fb9d317dc,"{'isPartial': True, 'annotations': [{'label': ...",labeler32@cogitotech.com,8a1a3d87-8a2d-47e5-8670-337fb9d317dc,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['medium_turbidity'], 'crops': [], 'p...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",1,3,"[{'label': 'PARTIAL', 'width': 161, 'xCrop': 0...",/data8tb/biomass-retrain/Image00002.jpg,100,2020-07-13T16:25:59.040183000Z
3,77e5b95e-ebb9-4444-813d-3894a2452956,9288fc60-ba5b-4da3-a147-b45235ebc826,"{'isPartial': True, 'annotations': [{'label': ...",labeler39@cogitotech.com,9288fc60-ba5b-4da3-a147-b45235ebc826,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['low_turbidity', 'small_fish'], 'cro...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",5,25,"[{'label': 'PARTIAL', 'width': 208, 'xCrop': 3...",/data8tb/biomass-retrain/Image00003.jpg,173,2021-01-03T15:04:32.449639000Z
4,a1ab3c2e-dcef-423f-a041-01fd971c996e,57d6a0f8-2a1e-4587-8883-c6948a2ea069,"{'isPartial': True, 'annotations': [{'label': ...",labeler36@cogitotech.com,57d6a0f8-2a1e-4587-8883-c6948a2ea069,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['high_turbidity'], 'crops': [{'id': ...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",1,7,"[{'label': 'PARTIAL', 'width': 414, 'xCrop': 9...",/data8tb/biomass-retrain/Image00004.jpg,95,2020-06-22T09:57:03.681510000Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10748,26237944-1e11-4f4f-8667-dc9d0796f56b,409ac351-7017-4d1a-89f0-7d8a127aa192,{'skipReasons': ['is_dark']},labeler41@cogitotech.com,409ac351-7017-4d1a-89f0-7d8a127aa192,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['low_turbidity'], 'crops': [], 'pen_...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",0,0,[],/data8tb/biomass-retrain/Image10748.jpg,194,2021-01-03T16:19:24.080397000Z
10749,73161391-987b-472b-9243-2fd3a1932a63,e6691b4e-a676-4cf4-9049-7002a8d5be70,"{'isPartial': False, 'annotations': [{'label':...",labeler39@cogitotech.com,e6691b4e-a676-4cf4-9049-7002a8d5be70,s3://aquabyte-research/pwais/mft-pg/datasets_s...,"{'tags': ['medium_turbidity', 'trout'], 'crops...",00000000-0000-0000-0000-000000000112,fish_detection_v2,"{'labelSets': [{'name': 'FISH', 'type': 'bbox'...",5,6,"[{'label': 'MEDIUM', 'width': 214, 'xCrop': 18...",/data8tb/biomass-retrain/Image10749.jpg,86,2020-06-10T14:16:17.568247000Z
10750,f2220bfd-5eec-449c-bcc5-1a1f6f8b4420,23306b76-40e7-40b5-bc64-7c9c14581db0,"{'isPartial': T

In [4]:
# #check if original annotation is the same as annotation, which is indeed the case
# df_hr_dataset['is_same'] = df_hr_dataset.apply(lambda x: x['original_annotation'].get('annotations', []) == x['annotation'], axis=1)

# # get subset of the dataset where original_annotation is not the same as annotation
# df_hr_dataset_not_same = df_hr_dataset[~df_hr_dataset['is_same']]
# df_hr_dataset_not_same

In [5]:
# download the images from images column to the local machine, save them in the IMAGE_SAVE_PATH
local_paths = []
not_found = []
is_val = []
rand = random.Random(1337)

for i, row in tqdm(df_hr_dataset.iterrows(), total=df_hr_dataset.shape[0]):
    is_val_image = rand.random() < VAL_FRAC
    remote_path = row['images']
    if is_val_image:
        local_path = f"{VAL_SAVE_PATH}{row.plali_image_id}.jpg"
        is_val.append(True)
    else:
        local_path = f"{TRAIN_SAVE_PATH}{row.plali_image_id}.jpg"
        is_val.append(False)
    bucket = remote_path.split('/')[2]
    key = '/'.join(remote_path.split('/')[3:])
    local_paths.append(local_path)
    # check if the file already exists
    if os.path.exists(local_path):
        continue
    # download the file, catch if the file is not found
    try:
        s3.download_file(bucket, key, local_path)
    except Exception as e:
        not_found.append(row.plali_image_id)
        print(e)
df_hr_dataset['local_path'] = local_paths
df_hr_dataset['is_val_image'] = is_val
# drop the rows where the image was not found
df_hr_dataset_found = df_hr_dataset[~df_hr_dataset['plali_image_id'].isin(not_found)]
df_hr_dataset_found = df_hr_dataset_found.reset_index(drop=True).copy()


 91%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊              | 9831/10753 [09:06<00:44, 20.88it/s]

An error occurred (404) when calling the HeadObject operation: Not Found


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10753/10753 [09:58<00:00, 17.96it/s]


In [6]:
# loop through the images and get the image height and width
image_heights = []
image_widths = []
for i, row in tqdm(df_hr_dataset_found.iterrows(), total=df_hr_dataset_found.shape[0]):
    local_path = row['local_path']
    im = Image.open(local_path)
    image_heights.append(im.height)
    image_widths.append(im.width)
df_hr_dataset_found['image_height'] = image_heights
df_hr_dataset_found['image_width'] = image_widths


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10752/10752 [00:02<00:00, 3997.50it/s]


In [7]:
# loop through df_hr_dataset_found.annotations and check each annotation has a label, width, height, xCrop, and yCrop, if there is a missing value, drop the dict
def clean_annotations(annotations:list):
    for annotation in annotations:
        if any([annotation.get('label') is None, annotation.get('width') is None, annotation.get('height') is None, annotation.get('xCrop') is None, annotation.get('yCrop') is None]):
            print(f"dropping annotation {annotation} from {annotations} because it is missing a value")
            time.sleep(1)
            annotations.remove(annotation)
    return annotations

df_hr_dataset_found['annotation'] = df_hr_dataset_found['annotation'].apply(clean_annotations)
#save the dataframe to a csv file
df_hr_dataset_found.to_csv(f"{ROOT_DIR}hrf_with_keypoint_visibility_dataset.csv", index=False)

dropping annotation {'label': 'PARTIAL', 'category': 'FISH'} from [{'label': 'PARTIAL', 'width': 223, 'xCrop': 216, 'yCrop': 154, 'height': 99, 'category': 'FISH'}, {'label': 'PARTIAL', 'width': 191, 'xCrop': 0, 'yCrop': 0, 'height': 93, 'category': 'FISH'}, {'label': 'PARTIAL', 'width': 156, 'xCrop': 355, 'yCrop': 117, 'height': 107, 'category': 'FISH'}, {'label': 'MEDIUM', 'width': 195, 'xCrop': 236, 'yCrop': 331, 'height': 77, 'category': 'FISH'}, {'label': 'PARTIAL', 'width': 140, 'xCrop': 60, 'yCrop': 48, 'height': 92, 'category': 'FISH'}, {'label': 'MEDIUM', 'width': 165, 'xCrop': 221, 'yCrop': 100, 'height': 75, 'category': 'FISH'}, {'label': 'MEDIUM', 'width': 153, 'xCrop': 343, 'yCrop': 451, 'height': 60, 'category': 'FISH'}, {'label': 'MEDIUM', 'width': 148, 'xCrop': 154, 'yCrop': 287, 'height': 58, 'category': 'FISH'}, {'label': 'PARTIAL', 'width': 93, 'xCrop': 419, 'yCrop': 205, 'height': 86, 'category': 'FISH'}, {'label': 'PARTIAL', 'width': 139, 'xCrop': 192, 'yCrop': 245

In [8]:
# adapted from https://github.com/aquabyte-new/research-exploration/blob/master/pwais/mft-pg/mft_utils/coco_dataset.py
annos_val = []
images_val = []
annotations = []
images = []
val_images = []

for _, row in df_hr_dataset_found.iterrows():
    is_val_image = row.is_val_image
    img_idx = len(val_images) if is_val_image else len(images)
    for bbox in row.annotation:
        anno_id = len(annos_val) + 1 if is_val_image else len(annotations) + 1
        category_id = category_to_id[bbox['label']]
        bbox_x, bbox_y, bbox_w, bbox_h = bbox['xCrop'], bbox['yCrop'], bbox['width'], bbox['height']
        anno = {
            "id": anno_id,
            "image_id": img_idx,
            "category_id": category_id,
            "bbox": [bbox_x, bbox_y, bbox_w, bbox_h],
            # "keypoints": [],
            # "num_keypoints": 0,
            "score": -1,
            "area": bbox_w * bbox_h,
            "iscrowd": 0,
        }
        if is_val_image:
            annos_val.append(anno)
        else:
            annotations.append(anno)

        img_path = row.local_path
        img_fname = os.path.basename(img_path)

        image = {
            "id": img_idx,
            "file_name": img_fname,
            "height": row.image_height,
            "width": row.image_width,
        }
        if is_val_image:
            val_images.append(image)
        else:
            images.append(image)
            
categories = [
{'id': id_, 'name': name}
for name, id_ in category_to_id.items()
]

train_json = {
'categories': categories,
'images': images,
'annotations': annotations,
}

val_json = {
'categories': categories,
'images': val_images,
'annotations': annos_val,
}

with open(f"{ANNOTATION_SAVE_PATH}train.json", "w") as f:
    json.dump(train_json, f)

with open(f"{ANNOTATION_SAVE_PATH}val.json", "w") as f:
    json.dump(val_json, f)